In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf

In [2]:
import glob
uav_path = '../data/phantom/JUNE_01_PHANTOMS/wavs/*.wav'
none_path = '../data/phantom/JUNE_02_BACKGROUND/wavs/background/background_0*.wav'
uav_files = glob.glob(uav_path)
none_files = glob.glob(none_path)
#print(uav_files)
#print(none_files)

In [12]:
CHUNK_SIZE = 8192
SR = 44100
N_MFCC = 16

In [4]:
def load(files, sr=44100):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
uav_raw = load(uav_files)
none_raw = load(none_files)

(9732096,)
(18579456,)


### Data preprocessing

Chose `mfcc4` among functions below:
- input size 16x16; `n_mfcc=16`, used first 16 data points from 16x17 mfcc of a chunk with size 8192
- chunk siez 8192, overlapped half of it

In [6]:
def mfcc(raw, chunk_size=8192, sr=44100, n_mfcc=13):
    mfcc = np.empty((13, 0))
    for i in range(0, len(raw), chunk_size):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc)
        mfcc = np.hstack((mfcc, mfcc_slice))
    return mfcc

In [8]:
def mfcc2(raw, label, chunk_size=8192, sr=44100, n_mfcc=13):
    mfcc = np.empty((0, n_mfcc, 17))
    y = []
    for i in range(0, len(raw), chunk_size):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc)
        mfcc_slice = mfcc_slice.reshape((1,n_mfcc,17))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [10]:
def mfcc3(raw, label, chunk_size=8192, sr=44100, n_mfcc=13):
    mfcc = np.empty((n_mfcc*17, 0))
    y = []
    for i in range(0, len(raw), chunk_size):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc)
        #print(mfcc_slice)
        mfcc_slice = mfcc_slice.reshape((n_mfcc*17, 1)) #mfcc_slice.reshape((1,n_mfcc,17))
        #print(mfcc_slice)
        mfcc = np.hstaack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=44100, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
mfcc_uav, y_uav = mfcc4(uav_raw, 1)
print(mfcc_uav.shape, y_uav.shape)
mfcc_none, y_none = mfcc4(none_raw, 0)
print(mfcc_none.shape, y_none.shape)


(9732096,)
small: (16, 9)
end
(2375, 16, 16) (2375,)
(18579456,)
small: (16, 9)
end
(4535, 16, 16) (4535,)


In [8]:
print(mfcc_uav.shape, y_uav.shape)
print(mfcc_none.shape, y_none.shape)

(2375, 16, 16) (2375,)
(4535, 16, 16) (4535,)


In [9]:
X = np.concatenate((mfcc_uav, mfcc_none), axis=0)
y = np.hstack((y_uav, y_none))
print(X.shape, y.shape)

(6910, 16, 16) (6910,)


In [10]:
n_labels = y.shape[0]
n_unique_labels = 2
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(6910, 2)


In [11]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [5]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(5528, 16, 16) (1382, 16, 16)
(5528, 2) (1382, 2)


In [54]:
np.save('../data/Xy/X_train_chunk_3d', X_train)
np.save('../data/Xy/X_test_chunk_3d', X_test)
np.save('../data/Xy/y_train_chunk_3d', y_train)
np.save('../data/Xy/y_test_chunk_3d', y_test)

In [2]:
X_train = np.load('../data/Xy/X_train_chunk_3d.npy')
X_test = np.load('../data/Xy/X_test_chunk_3d.npy')
y_train = np.load('../data/Xy/y_train_chunk_3d.npy')
y_test = np.load('../data/Xy/y_test_chunk_3d.npy')

# Tensorflow finally!

## Training

### Parameters
- input data: ?, 16, 16, 1 (batch size, #mfcc, #frame, #channels)
- filter: [3, 3]
- strides: 1(first layer), 2(second layer)

### Experiment 1 - One convolutional layer

In [ ]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])


conv1 = tf.layers.conv2d(inputs=X, filters=1, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=1)
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=True)

flat = tf.reshape(dropout1, [-1, 16*16*1])


In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [ ]:
batch_size = 1#y_train.shape[0]
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(y_train.shape[0] / batch_size)
    for i in range(0, y_train.shape[0], batch_size):
        feed_dict={X:X_train2[i:i+batch_size,:,:,:], Y:y_train[i:i+batch_size,:]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c/total_batch
    print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))

In [ ]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test2})
y_true = sess.run(tf.argmax(y_test,1))

#### Experiment 1-1
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- Question: cost increased overall with a little fluctuation during training. (3.16 -> 6.19)

In [40]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.975
Accuracy:  0.9746743849493488
             precision    recall  f1-score   support

          0       0.97      0.99      0.98       885
          1       0.98      0.95      0.96       497

avg / total       0.97      0.97      0.97      1382

[[875  10]
 [ 25 472]]


#### Experiment 1-2
- learning rate 0.01
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [24]:
# learning_rate 0.01, pooling stride 1x1, dropout, cost 0.8 --> 1.3 ....why?
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.968
Accuracy:  0.9681620839363242
             precision    recall  f1-score   support

          0       0.99      0.96      0.97       885
          1       0.94      0.97      0.96       497

avg / total       0.97      0.97      0.97      1382

[[854  31]
 [ 13 484]]


#### Experiment 1-3
- pooling stride 2x2
- #filter 1
- used code below for the graph
- 

In [ ]:
#X = tf.placeholder(tf.float32, shape=[None,n_mfcc,n_frame,n_channels])
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

W1 = tf.Variable(tf.random_normal([kernel_size, kernel_size, n_channels, n_filters], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding=pad)
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,1,1,1], padding=pad)
L1 = tf.reshape(L1, [-1, 16*16*1])

In [ ]:
# fully connected layer
W2 = tf.get_variable("W2", shape=[16*16*1,n_classes])
initializer = tf.contrib.layers.xavier_initializer()

In [ ]:
b = tf.Variable(tf.random_normal([n_classes]))
logits = tf.matmul(L1, W2) + bb

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [12]:
# learning rate 0.005 epoch 20
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: X_test2, Y: y_test}))

Accuracy: 0.9218524


In [12]:
# epoch 20
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: X_test2, Y: y_test}))

Accuracy: 0.97178


In [13]:
# epoch 10
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: X_test2, Y: y_test}))

Accuracy: 0.9652677


### Experiment 2 - Two convolutional layers

In [19]:
tf.reset_default_graph()

In [3]:
n_mfcc = 16
n_frame = 16
n_classes = 2
n_channels = 1

kernel_size = 3
stride = 1
pad = "SAME"

learning_rate = 0.01
training_epochs = 20

In [4]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels])
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

In [5]:
conv1 = tf.layers.conv2d(inputs=X, filters=5, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=1)
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=True)


conv2 = tf.layers.conv2d(inputs=dropout1, filters=5, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                padding="SAME", strides=2)
dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7, training=True)


In [13]:
print(dropout2)

Tensor("dropout_1/dropout/mul:0", shape=(?, 8, 8, 5), dtype=float32)


In [14]:
flat = tf.reshape(dropout2, [-1,  8*8*5])
dense3 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
dropout3 = tf.layers.dropout(inputs=dense3, rate=0.5, training=True)
logits = tf.layers.dense(inputs=dropout3, units=2)

In [16]:
print(logits)

Tensor("add:0", shape=(?, 2), dtype=float32)


In [15]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [18]:
batch_size = 1 # no batch set yet!
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(y_train.shape[0] / batch_size)
    for i in range(0, y_train.shape[0], batch_size):
        feed_dict={X:X_train2[i:i+batch_size,:,:,:], Y:y_train[i:i+batch_size,:]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c/total_batch
    print('Epoch:', '%04d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost =  3.027617059
Epoch: 0002 cost =  0.663639208
Epoch: 0003 cost =  0.643209847
Epoch: 0004 cost =  0.643210008
Epoch: 0005 cost =  0.643210009
Epoch: 0006 cost =  2.863240837
Epoch: 0007 cost =  0.643214880
Epoch: 0008 cost =  0.643210005
Epoch: 0009 cost =  0.643210009
Epoch: 0010 cost =  0.643210009
Epoch: 0011 cost =  0.643210009
Epoch: 0012 cost =  0.643210009
Epoch: 0013 cost =  0.643210009


KeyboardInterrupt: 

In [19]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test2})
y_true = sess.run(tf.argmax(y_test,1))

#### Experiment 2-1
- learning rate 0.01
- pooling stride 1x1
- #filter 5 for each conv layer
- detects everything as no uav

In [20]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.64
Accuracy:  0.6403762662807525
             precision    recall  f1-score   support

          0       0.64      1.00      0.78       885
          1       0.00      0.00      0.00       497

avg / total       0.41      0.64      0.50      1382

[[885   0]
 [497   0]]


/Users/bowonyang/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Experiment 2-2
- learning rate 0.005
- pooling stride 1x1
- #filter 5 for each conv layer
- detects everything as no uav

In [18]:
# cost history 5.75->0.64... but low f1 score (overfitting???)
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.64
Accuracy:  0.6403762662807525
             precision    recall  f1-score   support

          0       0.64      1.00      0.78       885
          1       0.00      0.00      0.00       497

avg / total       0.41      0.64      0.50      1382

[[885   0]
 [497   0]]


/Users/bowonyang/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Experiment 2-3
- learning rate 0.01
- pooling stride 1x1
- #filter 10 for each conv layer
- detects everything as no uav

In [18]:
# learning_rate 0.01, pooling stride 1x1, dropout, filter 10
# 2 layers
# cost 5.75->0.64... but low f1 score (overfitting???)
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.64
Accuracy:  0.6403762662807525
             precision    recall  f1-score   support

          0       0.64      1.00      0.78       885
          1       0.00      0.00      0.00       497

avg / total       0.41      0.64      0.50      1382

[[885   0]
 [497   0]]


/Users/bowonyang/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
